## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-18-13-03-50 +0000,bbc,What do we know about the Epstein files?,https://www.bbc.com/news/articles/c20r07dg6kro...
1,2025-11-18-13-01-48 +0000,nypost,Hochul to announce prison closure Tuesday as g...,https://nypost.com/2025/11/18/us-news/hochul-t...
2,2025-11-18-13-00-00 +0000,wsj,Nexstar CEO Says Bigger Is Better in Pitch to FCC,https://www.wsj.com/business/media/nexstar-ceo...
3,2025-11-18-12-59-38 +0000,nyt,Why Britain Is Embracing ‘Negative Nation Bran...,https://www.nytimes.com/2025/11/17/world/europ...
4,2025-11-18-12-50-45 +0000,bbc,US House expected to approve resolution to rel...,https://www.bbc.com/news/articles/cn0gdzrklr7o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,45
1,epstein,18
187,new,15
2,files,13
21,house,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2025-11-18-12-00-00 +0000,wsj,Once one of President Trump’s most strident MA...,https://www.wsj.com/politics/policy/how-marjor...,128
225,2025-11-17-21-29-00 +0000,wsj,Trump said he would sign legislation to releas...,https://www.wsj.com/politics/policy/trump-says...,124
156,2025-11-18-02-00-00 +0000,wsj,President Trump’s grip on the GOP is showing s...,https://www.wsj.com/politics/policy/trumps-gri...,100
159,2025-11-18-01-48-49 +0000,nyt,Trump Has the Power to Release the Epstein Fil...,https://www.nytimes.com/2025/11/17/us/politics...,99
16,2025-11-18-12-38-40 +0000,bbc,When is the Epstein files vote - and why does ...,https://www.bbc.com/news/articles/cq50nvq8znvo...,94


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,128,2025-11-18-12-00-00 +0000,wsj,Once one of President Trump’s most strident MA...,https://www.wsj.com/politics/policy/how-marjor...
15,52,2025-11-18-12-39-21 +0000,nypost,House Democrat pushing for release of full Eps...,https://nypost.com/2025/11/18/us-news/rep-ro-k...
124,50,2025-11-18-06-07-06 +0000,nypost,LA County Sheriff investigating new sexual bat...,https://nypost.com/2025/11/18/us-news/la-count...
253,41,2025-11-17-19-37-45 +0000,nyt,France Agrees to Sell Up to 100 Rafale Fighter...,https://www.nytimes.com/2025/11/17/world/europ...
242,41,2025-11-17-20-22-42 +0000,nypost,Ex-Disney star ripped for ‘demonic’ app that l...,https://nypost.com/2025/11/17/business/ex-disn...
266,39,2025-11-17-18-40-35 +0000,nypost,Bangladesh’s ousted PM Sheikh Hasina sentenced...,https://nypost.com/2025/11/17/world-news/bangl...
228,39,2025-11-17-21-22-00 +0000,wsj,Fed Vice Chair Philip Jefferson acknowledged t...,https://www.wsj.com/economy/central-banking/fe...
44,37,2025-11-18-11-33-48 +0000,bbc,Minister insists government committed to asylu...,https://www.bbc.com/news/articles/cvgd2vq4enzo...
284,37,2025-11-17-17-39-35 +0000,nypost,Trump-supporting grocer says he may shut down ...,https://nypost.com/2025/11/17/business/trump-s...
102,37,2025-11-18-09-18-41 +0000,wapo,U.N. Security Council approves Trump’s 20-poin...,https://www.washingtonpost.com/national-securi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
